<a href="https://colab.research.google.com/github/rkp74/Smart_Transportation_System/blob/main/Road_Sign_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras 
from keras.models import Sequential, Model
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Dropout,BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy.misc
from matplotlib.pyplot import imshow
import cv2
import os

In [ ]:
# Connecting Google Drive with Google Colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
datapath =  "/content/drive/My Drive/Smart_Transportation_System/Dataset/sign"

In [ ]:
#loads the data and performs preprocessing of image
#Contrast adjustment and class imbalance is handled
def load_data(path):
    images = []
    classes = []
    data = pd.read_csv(path)
    data = data.sample(frac=1).reset_index(drop=True)
    for i,row in data.iterrows():
        img_path = row['Path']
        img_class = row['ClassId']
        
        img = os.path.join(datapath,img_path)
        img = cv2.imread(img)
        img_resize = cv2.resize(img,(32,32),3) #resizes each img to size 32*32*3
        R,G,B = cv2.split(img_resize)          #splits R, G & B values of img
        R = cv2.equalizeHist(R)                #Contrast adjustment of R channel 
        G = cv2.equalizeHist(G)                #Contrast adjustment of G channel
        B = cv2.equalizeHist(B)                #Contrast adjustment of B channel
        new_img = cv2.merge((R,G,B))
        
        if i>0 and i%1000 == 0: 
            print("loaded: ",i," images")
        images.append(new_img)
        classes.append(img_class)
        
    imgs = np.array(images)
    labels = np.array(classes)
    
    return imgs, labels

In [ ]:
train_X, train_Y = pd.read_csv('gtsrb_Train.csv')
test_X, test_Y = pd.read_csv('gtsrb_Test.csv')

ValueError: ignored

In [ ]:
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

In [ ]:
def plot_signs(X, Y):
    for i in range(20):
        plt.subplot(5, 4, i+1)
        plt.tight_layout()
        plt.imshow(X[i], cmap='hsv')
        plt.title('Class:{}'.format(Y[i]))
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
plot_signs(train_X,train_Y)

In [ ]:
#Normalizing the data
train_X = train_X.astype('float32')/255.0
test_X = test_X.astype('float32')/255.0

train_Y = to_categorical(train_Y)
test_Y = to_categorical(test_Y)

class_totals = train_Y.sum(axis=0)
class_weights = class_totals.max() / class_totals

In [ ]:
import tensorflow
def model(width,height,depth,classes):
    model = Sequential()
    
    model.add(Conv2D(32,(5,5),activation='relu', input_shape=(width,height,depth)))              
    model.add(BatchNormalization())         
    model.add(MaxPooling2D((2,2), strides=(2,2)))
              
    model.add(Conv2D(48,(5,5),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2), strides=(2,2)))
              
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dropout(0.2))
              
    model.add(Dense(84, activation='relu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(classes, activation='softmax'))
    return model

In [ ]:
model = model(32,32,3,43)

In [ ]:
model.build()
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
            rotation_range=10, 
            width_shift_range=0.1, 
            height_shift_range=0.1, 
            zoom_range=0.1)
datagen.fit(train_X)

In [ ]:
# Set a learning rate annealer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                                patience=3, 
                                verbose=1, 
                                factor=0.2, 
                                min_lr=1e-6)

In [ ]:
model.fit_generator(datagen.flow(train_X, train_Y, batch_size=128), steps_per_epoch=len(train_X)/128, epochs=20, validation_data=(test_X, test_Y), 
                    class_weight=class_weights,
                    callbacks=[reduce_lr])